# Extracción de Libros de Ficción - TPv02

Este notebook extrae datos de libros de ficción (fantasía, ciencia ficción, etc.) desde múltiples APIs de manera paralela y ecológica.

**Características:**
- ✅ Configuración simple mediante variables
- ✅ Ejecución paralela de las 4 APIs
- ✅ Respeto a los límites de tasa (ecológico)
- ✅ DataFrames independientes por API
- ✅ Exportación automática a CSV con timestamp
- ✅ Carpeta de resultados organizada

In [1]:
# Importar librerías necesarias
import requests
import pandas as pd
import time
from datetime import datetime
import concurrent.futures
import os
import warnings

warnings.filterwarnings('ignore')
print("✓ Librerías importadas exitosamente")

✓ Librerías importadas exitosamente


## Configuración de Parámetros

In [2]:
# ==================== CONFIGURACIÓN ====================
# Modifica estos valores según tus necesidades

# Género a buscar (fantasy, science fiction, horror, mystery, etc.)
SEARCH_GENRE = "fantasy"

# Límite de libros a extraer por cada API
LIMIT_PER_API = 100

# Pausa entre peticiones (segundos) - Ajustar según la API
RATE_LIMITS = {
    "Open Library": 1.0,    # 1 segundo entre peticiones
    "Google Books": 1.0,    # 1 segundo entre peticiones
    "Archive.org": 1.5,     # 1.5 segundos (más conservador)
    "LibriVox": 1.0         # 1 segundo entre peticiones
}

# Carpeta de salida
OUTPUT_DIR = "resultados"

# ========================================================

# Crear carpeta de resultados si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Timestamp para los archivos
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

print(f"Configuración cargada:")
print(f"  - Género: '{SEARCH_GENRE}'")
print(f"  - Límite por API: {LIMIT_PER_API} libros")
print(f"  - Carpeta de salida: '{OUTPUT_DIR}/'")
print(f"  - Timestamp: {TIMESTAMP}")

Configuración cargada:
  - Género: 'fantasy'
  - Límite por API: 100 libros
  - Carpeta de salida: 'resultados/'
  - Timestamp: 20251101_011107


## Clases de Extracción por API

In [3]:
class OpenLibraryExtractor:
    """Extrae datos de Open Library"""
    
    def __init__(self, genre, limit, rate_limit):
        self.name = "Open Library"
        self.url = "https://openlibrary.org/search.json"
        self.genre = genre
        self.limit = limit
        self.rate_limit = rate_limit
        self.data = []
    
    def extract(self):
        print(f"\n[{self.name}] Iniciando extracción...")
        page_size = 100
        offset = 0
        
        while len(self.data) < self.limit:
            try:
                params = {
                    'q': f'subject:{self.genre}',
                    'limit': page_size,
                    'offset': offset,
                    'fields': 'title,author_name,first_publish_year,isbn,language,number_of_pages_median,publisher,subject'
                }
                
                response = requests.get(self.url, params=params, timeout=20)
                response.raise_for_status()
                result = response.json()
                
                docs = result.get('docs', [])
                if not docs:
                    break
                
                for doc in docs:
                    if len(self.data) >= self.limit:
                        break
                    self.data.append(doc)
                
                offset += len(docs)
                print(f"  [{self.name}] Extraídos: {len(self.data)}/{self.limit}")
                time.sleep(self.rate_limit)
                
            except Exception as e:
                print(f"  ✗ [{self.name}] Error: {e}")
                break
        
        print(f"✓ [{self.name}] Completado: {len(self.data)} libros")
        return self.to_dataframe()
    
    def to_dataframe(self):
        if not self.data:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.data)
        # Normalizar campos que son listas
        if 'author_name' in df.columns:
            df['author_name'] = df['author_name'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        if 'language' in df.columns:
            df['language'] = df['language'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        if 'subject' in df.columns:
            df['subject'] = df['subject'].apply(lambda x: ', '.join(x[:5]) if isinstance(x, list) else x)
        
        return df


class GoogleBooksExtractor:
    """Extrae datos de Google Books"""
    
    def __init__(self, genre, limit, rate_limit):
        self.name = "Google Books"
        self.url = "https://www.googleapis.com/books/v1/volumes"
        self.genre = genre
        self.limit = limit
        self.rate_limit = rate_limit
        self.data = []
    
    def extract(self):
        print(f"\n[{self.name}] Iniciando extracción...")
        page_size = 40
        start_index = 0
        
        while len(self.data) < self.limit:
            try:
                params = {
                    'q': f'subject:{self.genre}',
                    'maxResults': page_size,
                    'startIndex': start_index,
                    'printType': 'books'
                }
                
                response = requests.get(self.url, params=params, timeout=20)
                response.raise_for_status()
                result = response.json()
                
                items = result.get('items', [])
                if not items:
                    break
                
                for item in items:
                    if len(self.data) >= self.limit:
                        break
                    # Extraer volumeInfo y aplanarlo
                    volume_info = item.get('volumeInfo', {})
                    flat_item = {
                        'id': item.get('id'),
                        **volume_info
                    }
                    self.data.append(flat_item)
                
                start_index += len(items)
                print(f"  [{self.name}] Extraídos: {len(self.data)}/{self.limit}")
                time.sleep(self.rate_limit)
                
            except Exception as e:
                print(f"  ✗ [{self.name}] Error: {e}")
                break
        
        print(f"✓ [{self.name}] Completado: {len(self.data)} libros")
        return self.to_dataframe()
    
    def to_dataframe(self):
        if not self.data:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.data)
        # Normalizar campos que son listas
        if 'authors' in df.columns:
            df['authors'] = df['authors'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        if 'categories' in df.columns:
            df['categories'] = df['categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        return df


class ArchiveOrgExtractor:
    """Extrae datos de Archive.org"""
    
    def __init__(self, genre, limit, rate_limit):
        self.name = "Archive.org"
        self.url = "https://archive.org/advancedsearch.php"
        self.genre = genre
        self.limit = limit
        self.rate_limit = rate_limit
        self.data = []
    
    def extract(self):
        print(f"\n[{self.name}] Iniciando extracción...")
        page_size = 100
        page = 1
        
        while len(self.data) < self.limit:
            try:
                params = {
                    'q': f'collection:books AND mediatype:texts AND subject:{self.genre}',
                    'fl[]': ['title', 'creator', 'date', 'identifier', 'language', 'subject', 'description'],
                    'sort[]': 'downloads desc',
                    'rows': page_size,
                    'page': page,
                    'output': 'json'
                }
                
                response = requests.get(self.url, params=params, timeout=30)
                response.raise_for_status()
                result = response.json()
                
                docs = result.get('response', {}).get('docs', [])
                if not docs:
                    break
                
                for doc in docs:
                    if len(self.data) >= self.limit:
                        break
                    self.data.append(doc)
                
                page += 1
                print(f"  [{self.name}] Extraídos: {len(self.data)}/{self.limit}")
                time.sleep(self.rate_limit)
                
            except Exception as e:
                print(f"  ✗ [{self.name}] Error: {e}")
                break
        
        print(f"✓ [{self.name}] Completado: {len(self.data)} libros")
        return self.to_dataframe()
    
    def to_dataframe(self):
        if not self.data:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.data)
        # Normalizar campos que son listas
        if 'subject' in df.columns:
            df['subject'] = df['subject'].apply(lambda x: ', '.join(x[:5]) if isinstance(x, list) else x)
        
        return df


class LibriVoxExtractor:
    """Extrae datos de LibriVox"""
    
    def __init__(self, genre, limit, rate_limit):
        self.name = "LibriVox"
        self.url = "https://librivox.org/api/feed/audiobooks"
        self.genre = genre
        self.limit = limit
        self.rate_limit = rate_limit
        self.data = []
    
    def extract(self):
        print(f"\n[{self.name}] Iniciando extracción...")
        
        try:
            params = {
                'format': 'json',
                'limit': 1000,  # Traer muchos para filtrar
                'extended': '1'
            }
            
            response = requests.get(self.url, params=params, timeout=30)
            response.raise_for_status()
            result = response.json()
            
            all_books = result.get('books', [])
            
            # Filtrar por género
            for book in all_books:
                if len(self.data) >= self.limit:
                    break
                
                genres = [g.get('name', '').lower() for g in book.get('genres', [])]
                if self.genre.lower() in genres:
                    # Aplanar autores
                    authors = book.get('authors', [])
                    book['author_name'] = authors[0].get('display_name', 'Unknown') if authors else 'Unknown'
                    
                    # Aplanar géneros
                    book['genres_str'] = ', '.join([g.get('name', '') for g in book.get('genres', [])])
                    
                    self.data.append(book)
            
            print(f"  [{self.name}] Extraídos: {len(self.data)}/{self.limit}")
            
        except Exception as e:
            print(f"  ✗ [{self.name}] Error: {e}")
        
        print(f"✓ [{self.name}] Completado: {len(self.data)} libros")
        return self.to_dataframe()
    
    def to_dataframe(self):
        if not self.data:
            return pd.DataFrame()
        
        return pd.DataFrame(self.data)


print("✓ Clases de extracción definidas")

✓ Clases de extracción definidas


## Función de Extracción Paralela

In [4]:
def extract_from_api(extractor_class, genre, limit, rate_limit):
    """Función auxiliar para ejecutar la extracción de forma paralela"""
    extractor = extractor_class(genre, limit, rate_limit)
    return extractor.name, extractor.extract()


def extract_all_parallel():
    """Ejecuta la extracción de todas las APIs en paralelo"""
    
    print("="*60)
    print("INICIANDO EXTRACCIÓN PARALELA")
    print("="*60)
    
    # Configurar extractores
    extractors = [
        (OpenLibraryExtractor, SEARCH_GENRE, LIMIT_PER_API, RATE_LIMITS["Open Library"]),
        (GoogleBooksExtractor, SEARCH_GENRE, LIMIT_PER_API, RATE_LIMITS["Google Books"]),
        (ArchiveOrgExtractor, SEARCH_GENRE, LIMIT_PER_API, RATE_LIMITS["Archive.org"]),
        (LibriVoxExtractor, SEARCH_GENRE, LIMIT_PER_API, RATE_LIMITS["LibriVox"])
    ]
    
    results = {}
    
    # Ejecutar en paralelo
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(extract_from_api, *ext): ext for ext in extractors}
        
        for future in concurrent.futures.as_completed(futures):
            try:
                api_name, dataframe = future.result()
                results[api_name] = dataframe
            except Exception as e:
                print(f"✗ Error en extracción: {e}")
    
    print("\n" + "="*60)
    print("EXTRACCIÓN COMPLETADA")
    print("="*60)
    
    return results


# Ejecutar extracción
api_dataframes = extract_all_parallel()

INICIANDO EXTRACCIÓN PARALELA

[Open Library] Iniciando extracción...

[Google Books] Iniciando extracción...

[Archive.org] Iniciando extracción...

[LibriVox] Iniciando extracción...
  [Archive.org] Extraídos: 4/100
  [Google Books] Extraídos: 10/100
  [Open Library] Extraídos: 100/100
✓ [Archive.org] Completado: 4 libros
  [Google Books] Extraídos: 20/100
✓ [Open Library] Completado: 100 libros
  [Google Books] Extraídos: 30/100
  [Google Books] Extraídos: 40/100
  [Google Books] Extraídos: 50/100
  [Google Books] Extraídos: 60/100
  [Google Books] Extraídos: 70/100
  [Google Books] Extraídos: 80/100
  [Google Books] Extraídos: 90/100
  [Google Books] Extraídos: 100/100
✓ [Google Books] Completado: 100 libros
  [LibriVox] Extraídos: 0/100
✓ [LibriVox] Completado: 0 libros

EXTRACCIÓN COMPLETADA


## Exportación de Resultados

In [5]:
def export_results(dataframes_dict, output_dir, timestamp):
    """Exporta cada DataFrame a un archivo CSV independiente"""
    
    print("\n" + "="*60)
    print("EXPORTANDO RESULTADOS")
    print("="*60)
    
    summary = []
    
    for api_name, df in dataframes_dict.items():
        if df.empty:
            print(f"⚠ [{api_name}] Sin datos para exportar")
            continue
        
        # Crear nombre de archivo seguro
        safe_name = api_name.replace(" ", "_").replace(".", "")
        filename = f"{safe_name}_{timestamp}.csv"
        filepath = os.path.join(output_dir, filename)
        
        # Exportar
        df.to_csv(filepath, index=False, encoding='utf-8')
        
        print(f"✓ [{api_name}] Exportado: {filename}")
        print(f"  - Registros: {len(df)}")
        print(f"  - Columnas: {len(df.columns)}")
        
        summary.append({
            'API': api_name,
            'Archivo': filename,
            'Registros': len(df),
            'Columnas': len(df.columns)
        })
    
    # Crear reporte de resumen
    report_file = os.path.join(output_dir, f"resumen_{timestamp}.txt")
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("REPORTE DE EXTRACCIÓN DE LIBROS\n")
        f.write("="*50 + "\n\n")
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Género buscado: {SEARCH_GENRE}\n")
        f.write(f"Límite por API: {LIMIT_PER_API}\n\n")
        f.write("RESULTADOS POR API:\n")
        f.write("-"*50 + "\n")
        
        for item in summary:
            f.write(f"\n{item['API']}:\n")
            f.write(f"  Archivo: {item['Archivo']}\n")
            f.write(f"  Registros: {item['Registros']}\n")
            f.write(f"  Columnas: {item['Columnas']}\n")
        
        f.write("\n" + "="*50 + "\n")
        f.write(f"Total de APIs procesadas: {len(summary)}\n")
        f.write(f"Total de registros: {sum(item['Registros'] for item in summary)}\n")
    
    print(f"\n✓ Reporte de resumen creado: resumen_{timestamp}.txt")
    
    return summary


# Exportar resultados
summary = export_results(api_dataframes, OUTPUT_DIR, TIMESTAMP)


EXPORTANDO RESULTADOS
✓ [Archive.org] Exportado: Archiveorg_20251101_011107.csv
  - Registros: 4
  - Columnas: 7
✓ [Open Library] Exportado: Open_Library_20251101_011107.csv
  - Registros: 100
  - Columnas: 8
✓ [Google Books] Exportado: Google_Books_20251101_011107.csv
  - Registros: 100
  - Columnas: 23
⚠ [LibriVox] Sin datos para exportar

✓ Reporte de resumen creado: resumen_20251101_011107.txt


## Resumen Final

In [6]:
# Mostrar resumen final
print("\n" + "="*60)
print("RESUMEN FINAL")
print("="*60)

total_records = 0

for api_name, df in api_dataframes.items():
    records = len(df)
    total_records += records
    status = "✓" if records > 0 else "⚠"
    print(f"{status} {api_name}: {records} registros")

print(f"\n📊 Total de libros extraídos: {total_records}")
print(f"📁 Archivos guardados en: '{OUTPUT_DIR}/'")
print(f"🎯 Género buscado: '{SEARCH_GENRE}'")
print(f"\n✅ Proceso completado exitosamente")


RESUMEN FINAL
✓ Archive.org: 4 registros
✓ Open Library: 100 registros
✓ Google Books: 100 registros
⚠ LibriVox: 0 registros

📊 Total de libros extraídos: 204
📁 Archivos guardados en: 'resultados/'
🎯 Género buscado: 'fantasy'

✅ Proceso completado exitosamente


## Visualización de Muestra

A continuación se muestran las primeras filas de cada DataFrame para verificación.

In [7]:
# Mostrar muestra de cada DataFrame
for api_name, df in api_dataframes.items():
    if not df.empty:
        print(f"\n{'='*60}")
        print(f"MUESTRA: {api_name}")
        print(f"{'='*60}")
        print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
        print(f"\nPrimeras 3 filas:")
        display(df.head(3))
    else:
        print(f"\n⚠ {api_name}: Sin datos para mostrar")


MUESTRA: Archive.org
Dimensiones: 4 filas × 7 columnas

Primeras 3 filas:


,creator,date,description,identifier,language,subject,title
0,"Haddix, Margaret Peterson",2000-01-01T00:00:00Z,[Originally published: New York : Simon & Schu...,amonghiddenshado00marg_0,eng,"Science Fiction, Fantasy, & Magic, Children: G...",Among the hidden
1,Catherine Fisher,2010-01-01T00:00:00Z,NaN,incarceron00fish,eng,"Fantasy, Prisons -- Fiction, Prisoners -- Fict...",Incarceron
2,"Browne, Anthony, 1946-",1983-01-01T00:00:00Z,"Neglected by her busy father, a lonely young g...",gorilla00brow,eng,"Gorilla, Fathers and daughters, Fantasy",Gorilla



MUESTRA: Open Library
Dimensiones: 100 filas × 8 columnas

Primeras 3 filas:


,author_name,first_publish_year,isbn,language,number_of_pages_median,publisher,title,subject
0,George MacDonald,1850,"[9781492360094, 9798517858313, 9781978025394, ...","por, fre, spa, eng",196,"[Independently published, Project Gutenberg, O...",Phantastes,"Fairy tales, Scottish Fantasy fiction, Fiction..."
1,L. Frank Baum,1909,"[9781654157869, 9798482819449, 9798578004834, ...","eng, spa",139,"[Puffin, Indoeuropeanpublishing.com, Amereon L...",The Road to Oz,"Fairy tales, Juvenile Fiction, Fiction, Oz (Im..."
2,L. Frank Baum,1911,"[9798786169264, 9781548199555, 9781686153983, ...","eng, rus",130,"[BiblioBazaar, The Perfect Library, Wonder Pub...",The Sea Fairies,"Children's stories, Juvenile fiction, Classic ..."



MUESTRA: Google Books
Dimensiones: 100 filas × 23 columnas

Primeras 3 filas:


,id,title,authors,publisher,publishedDate,description,industryIdentifiers,readingModes,pageCount,printType,...,contentVersion,panelizationSummary,imageLinks,language,previewLink,infoLink,canonicalVolumeLink,subtitle,averageRating,ratingsCount
0,klGC6xroFOwC,The Magician's Nephew Color Gift Edition,C. S. Lewis,Zondervan,2003-11-04,The first book in The Chronicles of Narnia is ...,"[{'type': 'ISBN_13', 'identifier': '9780060530...","{'text': False, 'image': False}",122,BOOK,...,0.3.2.0.preview.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com.ar/books?id=klGC6xroFO...,http://books.google.com.ar/books?id=klGC6xroFO...,https://books.google.com/books/about/The_Magic...,NaN,NaN,NaN
1,x7gvAAAACAAJ,"Moomin, Mymble and Little My",Tove Jansson,Sort of Books,2001,Finnish artist Tove Jansson's Moomin stories h...,"[{'type': 'ISBN_10', 'identifier': '0953522741...","{'text': False, 'image': False}",0,BOOK,...,preview-1.0.0,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com.ar/books?id=x7gvAAAACA...,http://books.google.com.ar/books?id=x7gvAAAACA...,https://books.google.com/books/about/Moomin_My...,NaN,NaN,NaN
2,FD72ekYZqIkC,A Wizard of Earthsea,Ursula K. Le Guin,Houghton Mifflin Harcourt,2012,A boy grows to manhood while attempting to sub...,"[{'type': 'ISBN_13', 'identifier': '9780547851...","{'text': False, 'image': True}",267,BOOK,...,1.2.2.0.preview.1,"{'containsEpubBubbles': False, 'containsImageB...",{'smallThumbnail': 'http://books.google.com/bo...,en,http://books.google.com.ar/books?id=FD72ekYZqI...,http://books.google.com.ar/books?id=FD72ekYZqI...,https://books.google.com/books/about/A_Wizard_...,NaN,NaN,NaN



⚠ LibriVox: Sin datos para mostrar
